In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./datasets/ft_data.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ["将以下多条数据划分为：广告,高质量,其他 分类标准: 1. 广告: 意图为推销商品、鼓舞人购买的言论，包含有显式的广告格式的言论和隐式推销商品的言论。 例句: '看到这么多人都喜欢这个法兰绒毛毯你就知道有多舒服了?不管冷气房盖著看电视还是冬天披著窝在沙发也是大享受仅此这档免费送限量的限量的限量的[url]抢完就没啰～#维纳斯五盒就送给你#价值2980正货?', '#??DeSlim好产品高复购#菌菌需求大不大菌菌效果好不好看看我们张医生的诊所?好产品高复购???新加坡??DeSlim益生菌·纤维' 2. 高质量: 涉政、涉意识形态的言论，无论言论什么倾向，涉及即算此类。3.其他: 不属于广告和高质量的均视为其他 优先级：高质量＞广告＞其他",
  "将以下多条数据划分为：广告,高质量,其他 分类标准: 1. 广告: 意图为推销商品、鼓舞人购买的言论，包含有显式的广告格式的言论和隐式推销商品的言论。 例句: '看到这么多人都喜欢这个法兰绒毛毯你就知道有多舒服了?不管冷气房盖著看电视还是冬天披著窝在沙发也是大享受仅此这档免费送限量的限量的限量的[url]抢完就没啰～#维纳斯五盒就送给你#价值2980正货?', '#??DeSlim好产品高复购#菌菌需求大不大菌菌效果好不好看看我们张医生的诊所?好产品高复购???新加坡??DeSlim益生菌·纤维' 2. 高质量: 涉政、涉意识形态的言论，无论言论什么倾向，涉及即算此类。3.其他: 不属于广告和高质量的均视为其他 优先级：高质量＞广告＞其他",
  "将以下多条数据划分为：广告,高质量,其他 分类标准: 1. 广告: 意图为推销商品、鼓舞人购买的言论，包含有显式的广告格式的言论和隐式推销商品的言论。 例句: '看到这么多人都喜欢这个法兰绒毛毯你就知道有多舒服了?不管冷气房盖著看电视还是冬天披著窝在沙发也是大享受仅此这档免费送限量的限量的限量的[url]抢完就没啰～#维纳斯五盒就送给你#价值2980正货?', '#??DeSlim好产品高复购#菌菌需求大不大菌菌效果好不好看看我们张医生的诊所?好产品高复购???新加坡??DeSlim益生菌·纤维' 2. 高质量: 涉政、涉意识形态的言论，无论言论什么倾向，涉及即算此类。3.其他: 不属于广告和高质量的均视为其他 优先级：高质量＞广告＞其他"

In [4]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/LLM-Research/Meta-Llama-3-8B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tokenizer.pad_token, tokenizer.pad_token_id, tokenizer.eos_token_id

('<|eot_id|>', 128009, 128009)

In [6]:
def process_func(example):
    MAX_LENGTH = 888    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|start_header_id|>user<|end_header_id|>\n\n{example['instruction'] + example['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}<|eot_id|>", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/4057 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4057
})

In [8]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

<|start_header_id|>user<|end_header_id|>

将以下多条数据划分为：广告,高质量,其他 分类标准: 1. 广告: 意图为推销商品、鼓舞人购买的言论，包含有显式的广告格式的言论和隐式推销商品的言论。 例句: '看到这么多人都喜欢这个法兰绒毛毯你就知道有多舒服了?不管冷气房盖著看电视还是冬天披著窝在沙发也是大享受仅此这档免费送限量的限量的限量的[url]抢完就没啰～#维纳斯五盒就送给你#价值2980正货?', '#??DeSlim好产品高复购#菌菌需求大不大菌菌效果好不好看看我们张医生的诊所?好产品高复购???新加坡??DeSlim益生菌·纤维' 2. 高质量: 涉政、涉意识形态的言论，无论言论什么倾向，涉及即算此类。3.其他: 不属于广告和高质量的均视为其他 优先级：高质量＞广告＞其他做TikTok30天，个月收获21560?#带货#赚钱#电商#教程#短视频#TikTok小店#小黄车#热门 <|eot_id|><|start_header_id|>assistant<|end_header_id|>

广告<|eot_id|><|eot_id|>


In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'广告<|eot_id|><|eot_id|>'

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/LLM-Research/Meta-Llama-3-8B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [11]:
model.enable_input_require_grads()

In [12]:
model.dtype

torch.bfloat16

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'k_proj', 'down_proj', 'q_proj', 'v_proj', 'up_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/autodl-tmp/LLM-Research/Meta-Llama-3-8B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'k_proj', 'down_proj', 'q_proj', 'v_proj', 'up_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.26047588741133265


In [16]:
args = TrainingArguments(
    output_dir="./output/llama3_0808",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=8,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,3.021500


KeyboardInterrupt: 

In [ ]:
peft_model_id="./llama3_lora_0808"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)